In [1]:
import pandas as pd
import numpy as np
import pydataset
import sklearn.preprocessing
import matplotlib.pyplot as plt

import env
import wrangle

# pull data from sql
df = wrangle.get_data_from_sql()

# Exercises

Our scenario continues:

>As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of $5.00 per customer.

Create `split_scale.py` that will contain the functions that follow. Each scaler function should create the object, fit and transform both train and test. They should return the scaler, train dataframe scaled, test dataframe scaled. Be sure your indices represent the original indices from train/test, as those represent the indices from the original dataframe. Be sure to set a random state where applicable for reproducibility!

1. `split_my_data(X, y, train_pct)`


2. `standard_scaler()`


3. `scale_inverse()`


4. `uniform_scaler()`


5. `gaussian_scaler()`


6. `min_max_scaler()`


7. `iqr_robust_scaler()`


In [2]:
df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.8
2,0016-QLJIS,90.45,65,5957.9
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


#### I could just drop those rows

In [3]:
# Filter my dataframe

df2 = df[df.total_charges != ' ']

In [4]:
# Validate that total_charges all have values

df2[df2.total_charges == ' ']

,customer_id,monthly_charges,tenure,total_charges


In [5]:
# More validating...

df2.total_charges.value_counts(dropna=True).sort_index()

100.35     1
1004.35    1
1005.7     1
1006.9     1
1008.7     1
          ..
973.95     1
978        1
980.35     1
982.95     1
987.95     1
Name: total_charges, Length: 1677, dtype: int64

In [6]:
df2.total_charges = df2.total_charges.astype(float)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1685 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1685 non-null   object 
 1   monthly_charges  1685 non-null   float64
 2   tenure           1685 non-null   int64  
 3   total_charges    1685 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 65.8+ KB


In [8]:
df.tenure.value_counts().sort_index()

0      10
1       2
2       1
3       3
4       4
5       1
6       8
7       4
8      13
9       3
10      5
11      5
12      9
13     11
14      7
15      5
16      7
17      4
18      6
19      8
20      4
21      6
22      4
23     12
24     16
25      9
26      4
27     11
28      1
29      5
30     10
31      9
32      4
33     10
34      7
35     17
36      9
37     11
38     11
39     13
40      9
41     13
42     13
43     13
44      8
45     19
46     25
47     22
48     21
49     15
50     18
51     15
52     29
53     18
54     28
55     18
56     25
57     28
58     26
59     24
60     33
61     32
62     35
63     39
64     40
65     42
66     43
67     56
68     65
69     66
70     88
71    137
72    343
Name: tenure, dtype: int64

In [9]:
# Replace any tenures of 0 with 1

df.tenure.replace(0, 1, inplace=True)

In [10]:
# Validate my tenure count for value 1

df.tenure.value_counts().sort_index()

1      12
2       1
3       3
4       4
5       1
6       8
7       4
8      13
9       3
10      5
11      5
12      9
13     11
14      7
15      5
16      7
17      4
18      6
19      8
20      4
21      6
22      4
23     12
24     16
25      9
26      4
27     11
28      1
29      5
30     10
31      9
32      4
33     10
34      7
35     17
36      9
37     11
38     11
39     13
40      9
41     13
42     13
43     13
44      8
45     19
46     25
47     22
48     21
49     15
50     18
51     15
52     29
53     18
54     28
55     18
56     25
57     28
58     26
59     24
60     33
61     32
62     35
63     39
64     40
65     42
66     43
67     56
68     65
69     66
70     88
71    137
72    343
Name: tenure, dtype: int64

In [11]:
df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,
416,2520-SGTTA,20.00,1,
453,2775-SEFEE,61.90,1,
505,3115-CZMZD,20.25,1,
524,3213-VVOLG,25.35,1,
678,4075-WKNIU,73.35,1,
716,4367-NUYAO,25.75,1,
726,4472-LVYGI,52.55,1,
941,5709-LVOEQ,80.85,1,


In [12]:
# Replace the blank total_charges with the monthly_charge for tenure == 1

df.total_charges.replace(' ', df.monthly_charges, inplace=True)

In [13]:
# Validate my changes

df[df.tenure == 1]

,customer_id,monthly_charges,tenure,total_charges
188,1099-GODLO,20.35,1,20.35
234,1371-DWPAZ,56.05,1,56.05
416,2520-SGTTA,20.00,1,20
453,2775-SEFEE,61.90,1,61.9
505,3115-CZMZD,20.25,1,20.25
524,3213-VVOLG,25.35,1,25.35
678,4075-WKNIU,73.35,1,73.35
716,4367-NUYAO,25.75,1,25.75
726,4472-LVYGI,52.55,1,52.55
941,5709-LVOEQ,80.85,1,80.85


In [14]:
df.total_charges = df.total_charges.astype(float)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


## Create my wrangle_telco() function and a wrangle.py file to reflect the prep I want

- Again, all the hard work you did above will be the guts in your wrangle function making it very simple to acquire this data in the same way, making it a repeatable process, again and again.


- Remember, this is the basic process you will go through in projects to aquire and prep your data and create modules containing the functions you build along the way.

In [16]:
def wrangle_telco():
    df = wrangle.get_data_from_sql()
    df.tenure.replace(0, 1, inplace=True)
    df.total_charges.replace(' ', df.monthly_charges, inplace=True)
    df.total_charges = df.total_charges.astype(float)
    return df

In [17]:
# Validate I can call my function from the wrangle_telco module

wrangle_telco()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75
...,...,...,...,...
1690,9964-WBQDJ,24.40,71,1725.40
1691,9972-EWRJS,19.25,67,1372.90
1692,9975-GPKZU,19.75,46,856.50
1693,9993-LHIEB,67.85,67,4627.65


In [18]:
# Disco!

df = wrangle_telco()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


### End with a python file `wrangle.py` that contains the function, `wrangle_telco()`, that will acquire the data and return a dataframe cleaned with no missing values.

In [19]:
# Test importing and calling my function to get my prepped df

df = wrangle.wrangle_telco()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


In [21]:
df.tenure.value_counts(dropna=False).sort_index()

1      12
2       1
3       3
4       4
5       1
6       8
7       4
8      13
9       3
10      5
11      5
12      9
13     11
14      7
15      5
16      7
17      4
18      6
19      8
20      4
21      6
22      4
23     12
24     16
25      9
26      4
27     11
28      1
29      5
30     10
31      9
32      4
33     10
34      7
35     17
36      9
37     11
38     11
39     13
40      9
41     13
42     13
43     13
44      8
45     19
46     25
47     22
48     21
49     15
50     18
51     15
52     29
53     18
54     28
55     18
56     25
57     28
58     26
59     24
60     33
61     32
62     35
63     39
64     40
65     42
66     43
67     56
68     65
69     66
70     88
71    137
72    343
Name: tenure, dtype: int64

In [22]:
df.total_charges.value_counts(dropna=False).sort_index()

19.85      1
20.00      1
20.25      1
20.35      1
20.45      1
          ..
8547.15    1
8564.75    1
8594.40    1
8670.10    1
8672.45    1
Name: total_charges, Length: 1687, dtype: int64

### Looks like you know what you're doing